# Wepick 데이터를 twiceSpark1의 ES에 벌크 인서트

-  환경: ElasticSearch 
-  테스트 머신: twiceSpark1

In [1]:
import requests
import time
import urllib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import json
import elasticsearch
import csv
import pickle
from elasticsearch.helpers import parallel_bulk, bulk
import re
import glob
import os
from datetime import timezone, timedelta, datetime
from pymongo import MongoClient
import random
import numpy as np
import sys
import pandas as pd
import csv

In [2]:
### twiceSpark1: 10.102.50.47
es_url = 'twiceSpark1:9200'

In [3]:
es = elasticsearch.Elasticsearch(es_url)

### wepick_data

format:
deal_id, user_id, slot_id, timestamp, impression_time, label(always one)

In [4]:
data_path = r'd:\WMIND\temp\pos_dataset.csv'

In [5]:
def make_wepick_documents2(file_path):
    with open(file_path) as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            source = {}
            v, u, slot, imp_dt = int(row[0]), int(row[1]), int(row[2]), row[4]         
            source['u'] = u
            source['v'] = v
            source['slot'] = slot
            source['rgtime'] = pd.to_datetime(row[3], unit='ms').to_pydatetime()
            doc = {
                #"_op_type":'create',
                "_index":"wepick_seq",
                "_type":"doc",
                #"_id": u,
                "_source": source
            }
            yield(doc)

In [6]:
for x in make_wepick_documents2(data_path):
    print(x)
    break

{'_source': {'slot': 24, 'rgtime': datetime.datetime(2018, 3, 29, 10, 16, 2, 191000), 'v': 3473871, 'u': 10000015}, '_type': 'doc', '_index': 'wepick_seq'}


In [13]:
es.indices.get_mapping(index='wepick_seq')

{'wepick_seq': {'mappings': {'doc': {'properties': {'rgtime': {'type': 'date'},
     'slot': {'type': 'long'},
     'u': {'type': 'long'},
     'v': {'type': 'long'}}}}}}

In [14]:
start = time.time()
total_time = 0
bulk(es, make_wepick_documents2(data_path), request_timeout=100)
total_time += (time.time() - start)
print("Elapsed Time {} sec".format(total_time))

Elapsed Time 30801.94765663147 sec


In [9]:
res = es.search(index='wepick_seq', body={'query': {'match':{'u':'1000007'}}})

In [10]:
type(res['hits']['hits'][0]['_source']['hist'][0]['rgtime'])

KeyError: 'hist'

In [11]:
print(res)

{'timed_out': False, 'hits': {'hits': [{'_id': 'j9MkW2QBD5J0ujsh9LE1', '_source': {'slot': 1, 'u': 1000007, 'v': 3398754, 'rgtime': '2018-03-12T08:20:37.294000'}, '_score': 1.0, '_type': 'doc', '_index': 'wepick_seq'}, {'_id': 'kdMkW2QBD5J0ujsh9LE1', '_source': {'slot': 5, 'u': 1000007, 'v': 3389057, 'rgtime': '2018-03-13T15:43:56.579000'}, '_score': 1.0, '_type': 'doc', '_index': 'wepick_seq'}, {'_id': 'k9MkW2QBD5J0ujsh9LE1', '_source': {'slot': 4, 'u': 1000007, 'v': 3411797, 'rgtime': '2018-03-13T15:44:10.067000'}, '_score': 1.0, '_type': 'doc', '_index': 'wepick_seq'}, {'_id': 'mNMkW2QBD5J0ujsh9LE1', '_source': {'slot': 71, 'u': 1000007, 'v': 1590568, 'rgtime': '2018-03-23T13:04:03.368000'}, '_score': 1.0, '_type': 'doc', '_index': 'wepick_seq'}, {'_id': 'mdMkW2QBD5J0ujsh9LE1', '_source': {'slot': 78, 'u': 1000007, 'v': 3168382, 'rgtime': '2018-03-26T11:46:13.347000'}, '_score': 1.0, '_type': 'doc', '_index': 'wepick_seq'}, {'_id': 'nNMkW2QBD5J0ujsh9LE1', '_source': {'slot': 1, 'u':